In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import difflib
from difflib import SequenceMatcher
import os

/Users/quert/venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_pt = torch.load('./ptfile/train.pt')
test_pt = torch.load('./ptfile/test.pt')
val_pt = torch.load('./ptfile/val.pt')

In [6]:
len(test_pt)
# len(full_test_pt)

201

In [7]:
test_pt[0].keys()
# full_test_pt[0].keys()

dict_keys(['document', 'summary'])

In [10]:
# error_idx = []
# for idx in range(len(train_pt)):
#     tag = len(train_pt[idx]['document'].split('\n\n')) == len(train_pt[idx]['summary'].split('\n\n'))
#     error_idx.append(tag)

In [16]:
test_pt[5]['document'].split('.\n\n')[0].strip()

# full_test_pt[7]['document']

'Egypt–Russia relations (Russian: Российско-египетские отношения) refer to bilateral relations between Egypt and Russia. Diplomatic relations between the Soviet Union and Egypt were established on August 26, 1943. Egypt has an embassy in Moscow, while Russia has an embassy in Cairo and a consulate-general in Alexandria.'

In [9]:
test_pt[5]['summary']

'Egypt–Russia relations (Russian: Российско-египетские отношения) refer to bilateral relations between Egypt and Russia. Diplomatic relations between the Soviet Union and Egypt were established on August 26, 1943. Egypt has an embassy in Moscow, while Russia has an embassy in Cairo and a consulate-general in Alexandria.'

In [ ]:
# full_test_pt[7]['summary']
test_pt_ori[5]['summary']


In [13]:
too_much = []
for idx in range(len(full_test_pt)):
    if (len(full_test_pt[idx]['summary'].split('\n\n'))-len(full_test_pt[idx]['document'].split('\n\n'))) > 5:
        too_much.append(idx)

In [14]:
# len(full_test_pt[7]['summary'].split('\n\n')) - len(full_test_pt[7]['document'].split('\n\n'))
len(too_much)

2

In [32]:
# dict = {'un_matched_indices': too_much}  
       
# df = pd.DataFrame(dict) 
    
# # saving the dataframe
# df.to_csv('indices.csv') 

In [114]:
len(train_labeled_fix)

1602

In [ ]:
train_labeled_fix[12]['content']

In [ ]:
train_pt[12]['summary']

In [58]:
len(train_labeled_fix[1]['content'].split('#####')), len(train_pt[1]['summary'].split('\n\n'))

(99, 191)

In [ ]:
num_pars = []
for idx in range(len(train_pt)):
    # num_pars.append(len(train_pt[idx]['document']))
    if len(train_pt[idx]['document'])!=1:
        num_pars.append(idx)
# np.max(num_pars)

In [60]:
output_str = []
for idx in range(len(test_pt)):
    single_sent = test_pt[idx]['document'].split('.\n\n')
    num_articles = math.ceil(len(single_sent)/10) # might be 0 if #articles<10
    check_less=0
    articles = []
    # define idex
    if num_articles==0: # number of paragraphs < 10
        start_ids = [0]
    else:
        start_ids = [x*10 for x in range(num_articles)]
    s = '.\n\n'
    for id in range(len(start_ids)):
        if len(start_ids)==1:
            single_str = s.join(single_sent)
        else:
            try:
                if len(single_sent)>=start_ids[id] and len(single_sent)<start_ids[id+1]: 
                    single_sent_lst = single_sent[start_ids[id]:len(single_sent)]
                    single_str = s.join(single_sent_lst)
                if len(single_sent)>start_ids[id] and len(single_sent)>start_ids[id+1]: 
                    single_sent_lst = single_sent[start_ids[id]:start_ids[id+1]]
                    single_str = s.join(single_sent_lst)
            except:
                if len(single_sent)>start_ids[id]:
                    single_sent_lst = single_sent[start_ids[id]:len(single_sent)]
                    single_str = s.join(single_sent_lst)
                else:
                    single_str = str(single_sent[-1])
        articles.append(single_str)
    output_str.append(articles)

In [63]:
type(output_str[0])

list

In [58]:
wrap = []
src_strs = output_str.copy()

for idx in range(len(src_strs)):
    idx_content = {}
    idx_content['document'] = src_strs[idx]
    idx_content['summary'] = summ_clusters[idx]
    wrap.append(idx_content)
torch.save(wrap, 'wceptest_train.pt')

10

### Calculate the similarity between non-updated summary and updated summary

In [42]:
# Calculate the #paragraphs of each clusters except news trigger
lens_train_docs = [len(train_pt[idx]['document'].split('.\n\n')[:-1]) for idx in range(len(train_pt))]
lens_train_summs = [len(train_pt[idx]['summary'].split('.\n\n')) for idx in range(len(train_pt))]
print(np.min(lens_train_docs), np.max(lens_train_docs), np.mean(lens_train_docs), np.median(lens_train_docs))
print(np.min(lens_train_summs), np.max(lens_train_summs), np.mean(lens_train_summs), np.median(lens_train_summs))


1 13 2.460674157303371 2.0
1 13 2.404494382022472 2.0


In [64]:
ratios = []
for idx in range(len(train_pt)):
    doc = ' \n\n'.join(train_pt[idx]['document'].split('\n\n')[:-1]).replace('.  ', '. ') 
    summ = ' \n\n'.join(train_pt[idx]['summary'].split('\n\n')).replace('.  \n\n', '. \n\n')
    ratios.append(SequenceMatcher(None, doc, summ).ratio())
print(np.min(ratios), np.max(ratios), np.mean(ratios), np.median(ratios))

0.049062049062049064 1.0 0.9659677156343112 0.9989094757872705


In [66]:
# filter >0.990
filtered_ratios = [ratio for ratio in ratios if ratio<0.990]
np.mean(filtered_ratios)

0.8904776355007463

In [67]:
len(ratios), len(filtered_ratios)

(1602, 485)

### Split pt into src and tgt

In [223]:
%cd /Users/quert/Downloads/gcp_tmp/ptfile

/Users/quert/Downloads/gcp_tmp/ptfile


In [225]:
train_pt = torch.load('./train.pt')

In [226]:
# train_pt[0]['document'].replace('.\n\n', '. \n\n')
train_pt[0]['document']

'Hugo Rafael Chávez Frías (IPA: [uɰo rafael tʃaβes fɾias]) (born July 28, 1954) is the 53rd and current President of Venezuela. As the leader of the Bolivarian Revolution, Chávez promotes his vision of democratic socialism, Latin American integration, and anti-imperialism. He is also an ardent critic of neoliberal globalization and  US foreign policy. \\c\\cA career military officer, Chávez founded the leftist Fifth Republic Movement after orchestrating a failed 1992 coup détat against former president Carlos Andrés Pérez. Chávez was elected President in 1998 on promises of aiding Venezuelas poor majority, and was reelected in 2000. Domestically, Chávez has launched massive Bolivarian Missions, whose goals are to combat disease, illiteracy, malnutrition, poverty, and other social ills. Abroad, Chávez has acted against the Washington Consensus by supporting alternative models of economic development, and has advocated cooperation among the worlds poor nations, especially those

In [219]:
src = '. \c\c'.join(train_pt[0]['document'].replace('.\n\n', '. \n\n').split('. \n\n')[:-1])
src

'Hugo Rafael Chávez Frías (IPA: [uɰo rafael tʃaβes fɾias]) (born July 28, 1954) is the 53rd and current President of Venezuela. As the leader of the Bolivarian Revolution, Chávez promotes his vision of democratic socialism, Latin American integration, and anti-imperialism.  He is also an ardent critic of neoliberal globalization and  US foreign policy. \\c\\cA career military officer, Chávez founded the leftist Fifth Republic Movement after orchestrating a failed 1992 coup détat against former president Carlos Andrés Pérez.  Chávez was elected President in 1998 on promises of aiding Venezuelas poor majority, and was reelected in 2000. Domestically, Chávez has launched massive Bolivarian Missions, whose goals are to combat disease, illiteracy, malnutrition, poverty, and other social ills. Abroad, Chávez has acted against the Washington Consensus by supporting alternative models of economic development, and has advocated cooperation among the worlds poor nations, especially tho

In [220]:
tgt = train_pt[0]['summary'].replace('.\n\n', '. \\c\\c')
tgt

'Hugo Rafael Chávez Frías (IPA: ) (born July 28, 1954) is the 53rd and current President of Venezuela. As the leader of the "Bolivarian Revolution," Chávez promotes his vision of democratic socialism, Latin American integration, and anti-imperialism.  He is also an ardent critic of neoliberal globalization and  US foreign policy. \\c\\cA career military officer, Chávez founded the leftist Fifth Republic Movement after orchestrating a failed 1992 coup détat against former president Carlos Andrés Pérez.  Chávez was elected President in 1998 on promises of aiding Venezuelas poor majority, and was reelected in 2000. Domestically, Chávez has launched massive Bolivarian Missions, whose goals are to combat disease, illiteracy, malnutrition, poverty, and other social ills. Abroad, Chávez has acted against the Washington Consensus by supporting alternative models of economic development, and has advocated cooperation among the worlds poor nations, especially those in Latin America. \\

In [221]:
# srcs, tgts = [], []

# for idx in range(len(train_pt)):
#     # src = '. \c\c'.join(train_pt[idx]['document'].replace('.\n\n', '. \n\n').split('. \n\n')[:-1]).encode('utf-8', 'ignore').decode('utf-8')
#     src = '. \c\c'.join(train_pt[idx]['document'].replace('\n\n', '.\n\n').replace('..\n\n', '.\n\n').replace('.\n\n', '. \n\n').split('. \n\n')[:-1]).encode('utf-8', 'ignore').decode('utf-8')
#     # tgt = train_pt[idx]['summary'].replace('.\n\n', '. \\c\\c').replace('\n\n', '\\c\\c').encode('utf-8', 'ignore').decode('utf-8') 
#     tgt = train_pt[idx]['summary'].replace('\n\n', '.\n\n').replace('..\n\n', '.\n\n').replace('.\n\n', '. \n\n').replace('. \n\n', '. \\c\\c').encode('utf-8', 'ignore').decode('utf-8') 
#     srcs.append(src)
#     tgts.append(tgt)

In [239]:
srcs, tgts = [], []
for idx in range(len(train_pt)):
    srcs.append(' \\c\\c'.join(train_pt[idx]['document'].split(' \\c\\c')[:-1]))
    tgts.append(train_pt[idx]['summary'])

In [241]:
!pwd

/Users/quert/Downloads/gcp_tmp/ptfile


In [242]:
with open('train.src', 'w') as f:
    for line in srcs:
        f.write(line+'\n')
with open('train.tgt', 'w') as f:
    for line in tgts:
        f.write(line+'\n')

### Calculate the ROUGE without trigger existed

In [201]:
%cd /Users/quert/Downloads/gcp_tmp

/Users/quert/Downloads/gcp_tmp


In [210]:
from rouge import FilesRouge

files_rouge = FilesRouge()
hyp_path = '/Users/quert/Downloads/gcp_tmp/ptfile/train.src'
# hyp_path = 'aft_test_rouge_sec_txt.src'
# hyp_path = 'aft_test_rouge_txt.src'
ref_path = '/Users/quert/Downloads/gcp_tmp/ptfile/train.tgt'
scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
scores

{'rouge-1': {'r': 0.9602756499203132,
  'p': 0.9809746373851904,
  'f': 0.9680553407150487},
 'rouge-2': {'r': 0.9482270776940344,
  'p': 0.9702044215681309,
  'f': 0.9562910689671305},
 'rouge-l': {'r': 0.959834044346764,
  'p': 0.9805090578224864,
  'f': 0.9676219908796095}}

In [211]:
# Calculate BERTScore
from bert_score import score
# with open("gen_summ/gen_summ.test.tgt") as f:
# with open("aft_model_txt.src") as f:
with open('/Users/quert/Downloads/gcp_tmp/ptfile/train.src') as f:
    cands = [line.strip() for line in f]

with open("/Users/quert/Downloads/gcp_tmp/ptfile/train.tgt") as f:
    refs = [line.strip() for line in f]
P, R, F1 = score(cands, refs, lang='en', verbose=True)
print(f"System level F1 score: {F1.mean():.3f}")

Downloading: 100%|███████████████████████████████████████████████████████████████████████████████████| 482/482 [00:00<00:00, 190kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████| 899k/899k [00:01<00:00, 722kB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████████████████| 456k/456k [00:01<00:00, 439kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████████████████| 1.43G/1.43G [01:06<00:00, 21.3MB/s]
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTrain

calculating scores...
computing bert embedding.


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [14:44<00:00, 19.65s/it]


computing greedy matching.


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:04<00:00,  5.49it/s]


done in 889.06 seconds, 1.80 sentences/sec
System level F1 score: 0.991


### Clean the paragraph splitting tokens and rebuild pt file

In [195]:
pt_file = torch.load('./first/val.pt')

In [196]:
srcs, tgts = [], []

for idx in range(len(pt_file)):
    # src = '. \c\c'.join(train_pt[idx]['document'].replace('.\n\n', '. \n\n').split('. \n\n')[:-1]).encode('utf-8', 'ignore').decode('utf-8')
    src = '. \c\c'.join(pt_file[idx]['document'].replace('\n\n', '.\n\n').replace('..\n\n', '.\n\n').replace('.\n\n', '. \n\n').split('. \n\n')[:]).encode('utf-8', 'ignore').decode('utf-8').replace('.  ', '. ')
    # tgt = train_pt[idx]['summary'].replace('.\n\n', '. \\c\\c').replace('\n\n', '\\c\\c').encode('utf-8', 'ignore').decode('utf-8') 
    tgt = pt_file[idx]['summary'].replace('\n\n', '.\n\n').replace('..\n\n', '.\n\n').replace('.\n\n', '. \n\n').replace('. \n\n', '. \\c\\c').replace('. . \\c\\c', '. \\c\\c').encode('utf-8', 'ignore').decode('utf-8') 
    srcs.append(src)
    tgts.append(tgt)

In [197]:
wrap = []
for idx in range(len(pt_file)):
    idx_content = {}
    summ_str = pt_file[idx]['summary']
    idx_content['document'] = srcs[idx]
    idx_content['summary'] = tgts[idx]
    wrap.append(idx_content)
torch.save(wrap, './val_rebuild.pt')

In [198]:
rebuild_pt = torch.load('./val_rebuild.pt')
len(rebuild_pt)

192

In [199]:
rebuild_pt[2]['document']

'On March 29, 2021, Adam Toledo, a 13-year-old Mexican American boy, was fatally shot by Chicago Police Department officer Eric Stillman in the Little Village neighborhood on the West Side of Chicago. Bodycam footage of the shooting was released publicly on April 15. \\c\\cSeveral protests take place around the city after the video is released, including one that briefly shut down northbound Michigan Avenue and another at Union Park.'

### Calculate the #paragraphs

In [319]:
orig_train_pt = torch.load('./bs3_original/train.pt')
train_pt = torch.load('./bs3/train.pt')

In [320]:
len(orig_train_pt[1]['document'].split('\n\n')), len(orig_train_pt[1]['summary'].split('\n\n'))

(179, 191)

In [322]:
len(train_pt[1]['document'].split('\\c\\c')), len(train_pt[1]['summary'].split('\\c\\c'))

(179, 191)

In [327]:
lengths_orig = []
lengths = []
for idx in range(len(train_pt)):
    lengths.append(abs(len(train_pt[idx]['document'].split('\\c\\c'))-len(train_pt[idx]['summary'].split('\\c\\c'))))
    lengths_orig.append(abs(len(orig_train_pt[idx]['document'].split('\n\n'))-len(orig_train_pt[idx]['summary'].split('\n\n'))))
lengths==lengths_orig

True

### Extract the trigger from instances

In [371]:
train_pt = torch.load('bs3_original/train.pt')
test_pt = torch.load('bs3_original/test.pt')
val_pt = torch.load('bs3_original/val.pt')

In [362]:
# extract the trigger
with open('triggers_train.src', 'w') as f:
    for idx in range(len(train_pt)):
        trigger = train_pt[idx]['document'].split('\\c\\c')[-1]
        f.write(trigger+'\n')

In [373]:
# extract the trigger
with open('triggers_test.src', 'w') as f:
    for idx in range(len(test_pt)):
        trigger = test_pt[idx]['document'].split('\\c\\c')[-1]
        f.write(trigger+'\n')
# extract the trigger
with open('triggers_val.src', 'w') as f:
    for idx in range(len(val_pt)):
        trigger = val_pt[idx]['document'].split('\\c\\c')[-1]
        f.write(trigger+'\n')

### Calculate the edit actions from labeled data (data from GCP)

In [386]:
labeled_test = torch.load('bs3_original/test_labeled.pt')
train_without_trigger = torch.load('./train_labeled_without_trigger.pt')

In [389]:
train_without_trigger[0]['content']

'[KEEP] Hugo Rafael [KEEP] Chávez [KEEP] Frías [RM] (IPA: [uɰo rafael tʃaβes fɾias]) [RM] (born July 28, 1954) is the 53rd and current President of Venezuela. [ADD] (IPA: ) (born July 28, 1954) is the 53rd and current President of Venezuela. [RM] As the leader of the Bolivarian Revolution, Chávez promotes his vision of democratic socialism, Latin American integration, and anti-imperialism. [ADD] As the leader of the "Bolivarian Revolution," Chávez promotes his vision of democratic socialism, Latin American integration, and anti-imperialism. [KEEP] He is also an ardent critic of neoliberal globalization and  US foreign policy. [KEEP] \\c\\cA career military officer, Chávez founded the leftist Fifth Republic Movement after orchestrating a failed 1992 coup détat against former president Carlos Andrés [KEEP] Pérez. [KEEP] Chávez was elected President in 1998 on promises of aiding Venezuelas poor majority, and was reelected in 2000. [KEEP] Domestically, Chávez has launched massive

### Truncate pt file to 10 instances for debugging

In [484]:
test_pt = torch.load('bs3_original/test.pt')

In [485]:
srcs, tgts = [], []
for idx in range(10):
    srcs.append(test_pt[idx]['document'])
    tgts.append(test_pt[idx]['summary'])

In [487]:
wrap = []
for idx in range(len(srcs)):
    idx_content = {}
    idx_content['document'] = srcs[idx]
    idx_content['summary'] = tgts[idx]
    wrap.append(idx_content)
torch.save(wrap, './bs3_original/eva_par.pt')

* Feed `\\c\\c` as splitting token, and mark `</p><p>` in script, it would do well.


### Convert the pt file to csv for decoder

In [9]:
train_pt = torch.load("./ptfile/bs3_original/train.pt")
test_pt = torch.load("./ptfile/bs3_original/test.pt")
val_pt = torch.load("./ptfile/bs3_original/val.pt")

In [11]:
documents, summaries = [], []
for idx in range(len(train_pt)):
    document = train_pt[idx]["document"]
    summary = train_pt[idx]["summary"]
    documents.append(document)
    summaries.append(summary)
pd.DataFrame({"document": documents, "summary": summaries}).to_csv("./ptfile/same_secs_insert_labeled/train.csv")

In [7]:
!pwd

/Users/quert/Downloads/gcp_tmp


### Extract the positive instances from classification report for decoding

In [126]:
sub_csv = pd.read_csv("./ptfile/same_secs_insert_labeled/submission.csv")
test_csv = pd.read_csv("./ptfile/same_secs_insert_labeled/merged_updated_test.csv")

In [127]:
# Extract the positive instances
targets = sub_csv.target.values
instances = test_csv.paragraph.values
pos_ids = [idx for idx in range(len(targets)) if targets[idx]==1]

In [128]:
len(pos_ids), len(targets)-len(pos_ids)

(371, 10054)

In [129]:
pos_instances = [instances[idx] for idx in pos_ids]
pd.DataFrame({"document": pos_instances}).to_csv("./ptfile/same_secs_insert_labeled/pos_classification.csv")

### Merge the generated (updated) paragraphs with others

In [130]:
updated_ins = pd.read_csv("./ptfile/same_secs_insert_labeled/generated_paragraphs.csv").paragraph.values

In [131]:
merged_instances = instances.copy()
for idx in range(len(pos_ids)):
    merged_instances[pos_ids[idx]] = updated_ins[idx]

In [132]:
pd.DataFrame({"paragraph": merged_instances}).to_csv("./ptfile/same_secs_insert_labeled/updated_file.csv")

### Re-construct the paragraphs to its corresponded instances

In [133]:
nums_df = pd.read_csv("./ptfile/same_secs_insert_labeled/merged_numpar_test.csv")
updated_df = pd.read_csv("./ptfile/same_secs_insert_labeled/updated_file.csv")

In [134]:
nums_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   Unnamed: 0   168 non-null    int64
 1   Num of pars  168 non-null    int64
dtypes: int64(2)
memory usage: 2.8 KB


In [135]:
nums_df.rename(columns={'Num of pars': 'nums'}, inplace=True)
nums_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168 entries, 0 to 167
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Unnamed: 0  168 non-null    int64
 1   nums        168 non-null    int64
dtypes: int64(2)
memory usage: 2.8 KB


In [136]:
nums = list(nums_df.nums.values)

In [137]:
hyp_instances = []
start = 0
for num in nums:
    end = start + num - 1
    # sel_range = [start, end]
    txt_in_range = updated_df.iloc[start:end+1, 1].to_list()
    instance = ". ".join(txt_in_range)
    hyp_instances.append(instance)
    start += num

In [138]:
test_pt = torch.load("./ptfile/same_secs_insert_labeled/test_samesecs_labeled.pt")
ref_instances = []
for idx in range(len(test_pt)):
    ref_instances.append(test_pt[idx]["summary"])

In [139]:
assert len(hyp_instances)==len(ref_instances)

In [146]:
pd.DataFrame({"hyp": hyp_instances, "ref": ref_instances}).to_csv("./ptfile/same_secs_insert_labeled/final_merged.csv")

### Calculate the hyp and ref instances

In [147]:
# Split the merged file into non_updated and updated file
final_csv = pd.read_csv("./ptfile/same_secs_insert_labeled/final_merged.csv")
hyp_lst = list(final_csv.hyp.values)
ref_lst = list(final_csv.ref.values)

In [148]:
with open("./ptfile/same_secs_insert_labeled/final_hyp.src", "w") as f:
    for idx in range(len(final_csv)):
        instance = str(ref_lst[idx]).replace("\n", "\c")
        f.write(instance+"\n")

In [ ]:
from rouge import FilesRouge

files_rouge = FilesRouge()
hyp_path = './ptfile/same_secs_insert_labeled/final_hyp.src'
ref_path = './ptfile/same_secs_insert_labeled/final_ref.tgt'
scores = files_rouge.get_scores(hyp_path, ref_path, avg=True)
scores